In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ProgbarLogger
from keras.optimizers import SGD
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.layers import Input
from keras.callbacks import Callback
from sklearn.metrics import f1_score

import keras.backend as K

import numpy as np
import os 

### Set Up

In [ ]:
os.chdir("{}".format(training_dir))
K.set_image_dim_ordering('tf')

In [ ]:
batch_size = 32

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'train',
        target_size=(299, 299),
        batch_size=batch_size,
        shuffle=True,
        class_mode='categorical') 

validation_generator = test_datagen.flow_from_directory(
        'test',
        target_size=(299, 299),
        batch_size=batch_size,
        shuffle=True,
        class_mode='categorical')

### Training

In [ ]:
print(os.getcwd())
K.clear_session()

base_model = InceptionV3(weights='imagenet', include_top=False, 
                         input_tensor=Input(shape=(299, 299, 3)))
x = base_model.output
x = AveragePooling2D(pool_size=(8, 8))(x)
x = Dropout(.4)(x)
x = Flatten()(x)
predictions = Dense(2, init='glorot_uniform', W_regularizer=l2(.0005), 
                    activation='softmax')(x)

model = Model(input=base_model.input, output=predictions)
opt = SGD(lr=.01, momentum=.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', 
              metrics=['accuracy', 'mae'])

checkpointer = ModelCheckpoint(filepath='model4.{epoch:02d}-{val_loss:.2f}.hdf5', 
                               verbose=1, 
                               save_best_only=True)
#monitor = RemoteMonitor(root="0.0.0.0:9000")

def schedule(epoch):
    if epoch < 15:
        return .01
    elif epoch < 28:
        return .002
    else:
        return .0004
lr_scheduler = LearningRateScheduler(schedule)

model.fit_generator(train_generator,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator.filenames) // batch_size,
                    steps_per_epoch=len(train_generator.filenames) // batch_size,
                    epochs=50,
                    verbose=1,
                    callbacks=[lr_scheduler, checkpointer])

### Testing

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
#import seaborn as sns

import keras.backend as K
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, roc_curve, auc, classification_report
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
%matplotlib inline
np.random.seed(42)

In [ ]:
K.set_image_dim_ordering('tf')
os.chdir('{}'.format(model_dir))
model = load_model(filepath=os.path.join(os.getcwd(), "best_model.hdf5"))

os.chdir("{}".format(testing_dir))

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    "validation",
    target_size=(299, 299),
    batch_size=32,
    shuffle=False,
    class_mode='categorical'
)

In [ ]:
preds = model.predict_generator(test_generator, steps=1066 // 32, verbose=1)

In [ ]:
## Accuracy
sum([1 if np.argmax(x) == y  else 0 for x, 
     y in zip(preds, test_generator.classes[:1056])]) / len(preds)

In [ ]:
cnf_matrix = confusion_matrix(np.argmax(preds, axis=1), test_generator.classes[:1056])
tp, fp, fn, tn = cnf_matrix.ravel()
print("Sensitivity: ", tp/ (tp + fn))
print("Specificity: ", tn/(fp + tn))
fig = plt.figure(figsize=(8, 8))
plt.clf()
ax = fig.add_subplot(111)
ax.set_aspect(1)
res = ax.imshow(np.array(cnf_matrix), cmap=plt.cm.summer, 
                interpolation='nearest')

width, height = cnf_matrix.shape

for x in range(width):
    for y in range(height):
        ax.annotate(str(cnf_matrix[x][y]), xy=(y, x), 
                    horizontalalignment='center',
                    verticalalignment='center')

cb = fig.colorbar(res)
plt.xticks(range(width), ["Outlier", "Non-Outlier"])
plt.yticks(range(height), ["Outlier", "Non-Outlier"])
x_lab = plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix of Neural Net Performance")
plt.savefig("confusion_matrix.png", dpi=300,
           bbox_extra_artists=x_lab)

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(test_generator.classes[:1056], 
                                  preds[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
plt.figure(figsize=(8, 8))
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange', lw=lw,
         label='ROC Curve: Outlier (area = {0:0.4f})'
         ''.format(roc_auc[1]))
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.savefig("roc_curve.png", dpi=300)

In [ ]:
print(classification_report(np.eye(2)[test_generator.classes[:1056]], 
                            np.round(preds, decimals=0)))